In [3]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import ComplementNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from collections import defaultdict
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score, classification_report, roc_auc_score
from yellowbrick.model_selection import ValidationCurve
from yellowbrick.classifier import ROCAUC
import warnings
random_state=42

In [4]:
data = pd.read_csv("/Users/max/Quick Jupyter Notebooks/MMAI/MMAI 823 - Finance/Bankruptcy_data_Final.csv", header=0)

In [5]:
data_noNa = data.dropna()

In [ ]:
corr = data_noNa.corr()

In [ ]:
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)

In [6]:
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
scaled_data_noNan = data_noNa.copy()

In [7]:
# Scale values using min/max if there are only positive values, 
# and standard scaler if the minimum value is less than 0 to capture the negative values

# STRIKE THIS: NAIVE BAYS CAN ONLY HAVE NON-NEGATIVE VALUES

for (columnName, columnData) in scaled_data_noNan.iteritems():
    scaled_data_noNan[columnName] = minmax_scaler.fit_transform(columnData.values.reshape(-1,1))

X = scaled_data_noNan.iloc[:, :13]
y = scaled_data_noNan.iloc[:,-1:]
# Stratified Shuffle Split into Train/Test
sss = StratifiedShuffleSplit(test_size=0.2, random_state=random_state)
sss.get_n_splits(X, y)

for train_idx, test_idx in sss.split(X,y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx].values, y.iloc[test_idx].values

In [8]:
# Resampling

resampled = defaultdict()

# SMOTEN-NN
resampler = SMOTEENN(random_state=random_state)
X_rs, y_rs = resampler.fit_resample(X_train,y_train)
X_rs["BK"] = y_rs
resampled['SMOTEENN'] = X_rs

# SMOTE-Tomek
resampler = SMOTETomek(random_state=random_state)
X_rs, y_rs = resampler.fit_resample(X_train,y_train)
X_rs["BK"] = y_rs
resampled['SMOTETomek'] = X_rs

#BorderlineSMOTE
resampler = BorderlineSMOTE(random_state=random_state)
X_rs, y_rs = resampler.fit_resample(X_train,y_train)
X_rs["BK"] = y_rs
resampled['BorderlineSMOTE'] = X_rs

#ADASYN
resampler = ADASYN(random_state=random_state)
X_rs, y_rs = resampler.fit_resample(X_train,y_train)
X_rs["BK"] = y_rs
resampled['ADASYN'] = X_rs


/opt/anaconda3/envs/QuickPythonEnv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/QuickPythonEnv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/QuickPythonEnv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/QuickPythonEnv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was pass

## Assumptions
 - Naive Bayes classifiers assume that all predictors have equal effect on the outcome, however we are fairly confident that this is not the case

In [ ]:
# corr_normalized = scaled_data_noNan.corr()

# ax = sns.heatmap(
#     corr_normalized, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# )

In [9]:
model = ComplementNB()

In [10]:
# Baseline Model - No Sampling

model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Balanced Accuracy Score
rf_score = balanced_accuracy_score(y_test, predictions)
# AUC Score
rf_auc = roc_auc_score(y_test, predictions)

print('Benchmark Balanced Accuracy: ' + str(rf_score))
print('Benchmark AUC Score:' + str(rf_auc))
print(classification_report(y_test, predictions, digits=3))


Benchmark Balanced Accuracy: 0.7935300905552622
Benchmark AUC Score:0.7935300905552621
              precision    recall  f1-score   support

         0.0      0.999     0.755     0.860     16146
         1.0      0.020     0.832     0.038        95

    accuracy                          0.756     16241
   macro avg      0.509     0.794     0.449     16241
weighted avg      0.993     0.756     0.855     16241



/opt/anaconda3/envs/QuickPythonEnv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
for set in resampled:
    warnings.filterwarnings("ignore")

    X_train = resampled[set].iloc[:, :13]
    y_train = resampled[set].iloc[:,-1:].values.reshape(-1,1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    # Balanced Accuracy Score
    rf_score = balanced_accuracy_score(y_test.reshape(-1,1), predictions.reshape(-1,1))
    # AUC Score
    rf_auc = roc_auc_score(y_test.reshape(-1,1), predictions.reshape(-1,1))
    print('Resampled Using ' + str(set))
    print('Benchmark Balanced Accuracy: ' + str(rf_score))
    print('Benchmark AUC Score:' + str(rf_auc))
    print(classification_report(y_test.reshape(-1,1), predictions.reshape(-1,1), digits=3))

Resampled Using SMOTEENN
Benchmark Balanced Accuracy: 0.7938990918395953
Benchmark AUC Score:0.7938990918395954
              precision    recall  f1-score   support

         0.0      0.999     0.735     0.847     16146
         1.0      0.019     0.853     0.036        95

    accuracy                          0.736     16241
   macro avg      0.509     0.794     0.442     16241
weighted avg      0.993     0.736     0.842     16241

Resampled Using SMOTETomek
Benchmark Balanced Accuracy: 0.7938990918395953
Benchmark AUC Score:0.7938990918395954
              precision    recall  f1-score   support

         0.0      0.999     0.735     0.847     16146
         1.0      0.019     0.853     0.036        95

    accuracy                          0.736     16241
   macro avg      0.509     0.794     0.442     16241
weighted avg      0.993     0.736     0.842     16241

Resampled Using BorderlineSMOTE
Benchmark Balanced Accuracy: 0.7467242986693787
Benchmark AUC Score:0.7467242986693787
 